Library Imports

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

Device Config

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Transform

In [ ]:
transform = transforms.ToTensor()

Load MNIST DataSets

In [5]:
train_data = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 22.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 605kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 5.58MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.05MB/s]


CNN Model

In [6]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(32*7*7, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = x.view(-1, 32*7*7)
    x = torch.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = CNN().to(device)

Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Training Loop

In [8]:
epochs = 5
for epoch in range(epochs):
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    Loss = criterion(outputs, labels)

    optimizer.zero_grad()
    Loss.backward()
    optimizer.step()
  print(f'Epoch [{epoch+1}/{epochs}] Loss: {Loss.item(): .4f}')


Epoch [1/5] Loss:  0.0583
Epoch [2/5] Loss:  0.0125
Epoch [3/5] Loss:  0.0383
Epoch [4/5] Loss:  0.1173
Epoch [5/5] Loss:  0.0046


Testing

In [9]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total: .2f}%')

Test Accuracy:  98.15%
